## Oracle Docker Image Shared Functions
The following are a selection of shared functions used by various Juypter Notebooks that whilst useful don't need to included in the main notebooks. 

The section below simply import the needed modules for the helper functions

In [27]:
import docker
import os
import tarfile
from prettytable import PrettyTable
from IPython.display import HTML, display, Markdown
import humanize
import tempfile
import re

```list_images``` has the docker client passed to it and then lists installed images

In [12]:
def list_images(client):
    from prettytable import PrettyTable
    table = PrettyTable(["Tag", "id", "Size"])
    check_for_tags = lambda i: i.tags[0] if len(i.tags) != 0 else ''
    for image in client.images.list():
        table.add_row(
            [check_for_tags(image), image.short_id.replace('sha256:', ''), humanize.naturalsize(image.attrs['Size'])])
    display(HTML(table.get_html_string()))

```list_containers``` has the docker client passed to it and then lists any created docker containers

In [13]:
def list_containers(client):
    table = PrettyTable(["Name", "Short id", "Status", "IP Address"])
    table.align["Name"] = "l"
    for container in client.containers.list(all=True):
        table.add_row([container.name, container.short_id, container.status, container.attrs['NetworkSettings']['IPAddress']])
    display(HTML(table.get_html_string()))

```copy_to``` has parameters for a file path for a file on the host, a parameter for the destination path on the container and a parameter which the docker container to copy the file to.

In [14]:
def copy_to(src, dest, container):
    
    
    os.chdir(os.path.dirname(src))
    srcname = os.path.basename(src)
    tar = tarfile.open(src + '.tar', mode='w')
    try:
        tar.add(srcname)
    finally:
        tar.close()

    tar_data = open(src + '.tar', 'rb').read()
    container.put_archive(path=os.path.dirname(dest), data=tar_data)
#     os.remove(src)
    os.remove(src + '.tar')

```copy_string_to_file``` has parameters for a string, the file to put the string into (container) and the reference to the container

In [26]:
def copy_string_to_file(string, dest, container):
    
    filename = os.path.basename(dest)
    tempdir = tempfile.gettempdir()
    
    with open(os.path.join(tempdir,filename),'w') as tf:
        tf.write(string)
   
    os.chdir(tempdir)
    tar = tarfile.open(filename + '.tar', mode='w')
    try:
        tar.add(filename)
    finally:
        tar.close()

    tar_data = open(filename + '.tar', 'rb').read()
    container.put_archive(path=os.path.dirname(dest), data=tar_data)
    os.remove(filename + '.tar')
    os.remove(filename)

```create_and_run_script``` has parameters for a script to run, the local path of the script, the container remote path, the command to execute and a reference to the container. 

In [1]:
def create_and_run_script(script, script_local_path, script_target_path,  command, target):
    
    copy_to(script, script_local_path, target.name+":"+script_target_path)

    print(command)
    target.start()
    output = target.exec_run(command,
                              stderr=True,
                              stdout=True,
                              stream=True
                              )
    for line in output[1]:
        o_line = line.decode('UTF-8')
        print(o_line.strip())

```containter_exec``` has parameters for the container to execute the command on, the command to run, whether to output the commands stdout/stderr and whether to return immediately. 

In [16]:
def containter_exec(target, target_command, show_output=True, stream_val=True):
    
    target.start()
    output = target.exec_run(target_command,
                                  stderr=True,
                                  stdout=True,
                                  stream=stream_val
                                  )
    if show_output:
        for line in output[1]:
            o_line = line.decode('UTF-8')
            print(o_line.strip())

```containter_root_exec``` is the same as ```containter_exec``` but runs as containers root

In [17]:
def containter_root_exec(target, target_command, show_output=True, stream_val=True):
    
    target.start()
    output = target.exec_run(target_command,
                                  stderr=True,
                                  stdout=True,
                                  stream=stream_val,
                                  user='root',
                                  workdir='/'
                                  )
    if show_output:
        for line in output[1]:
            o_line = line.decode('UTF-8')
            print(o_line.strip())